# Lecture 12

## Solving PDEs with the finite element method

The procedure for solving PDEs with the finite element method is exactly the same as for global basis functions. However

* Basis functions are local
* We always use reference elements
* Matrices are assembled elementwise
* We will only use Lagrange polynomials

As always we work with a function space $V_N = \text{span}\{\psi_{j}\}_{j=0}^N$ and approximations

$$
u_N(x) = \sum_{j=0}^N \hat{u}_j \psi_j(x).
$$

The basis functions $\psi_i$ for the FEM were discussed in [lecture 10](https://matmek-4270.github.io/matmek4270-book/lecture10.html), where we also discussed finite element assembly of the mass matrix

$$
a_{ij} = (\psi_j, \psi_i) = \sum_{e=0}^{N_e-1} \int_{\Omega^{(e)}} \psi_j \psi_i d\Omega.
$$

In this lecture we move one step further and use the FEM to solve ordinary differential equations. Exactly like in [lecture 11](https://matmek-4270.github.io/matmek4270-book/lecture11.html) this means that we want to find $u_N \in V_N$ such that

$$
(\mathcal{R}_N, v) = 0, \quad \forall \, v \in V_N,
$$

where $\mathcal{R}_N = \mathcal{L}(u_N)-f$ is a residual and $\mathcal{L}(u)$ is some mathematical operator acting on $u$. See [lecture 11](https://matmek-4270.github.io/matmek4270-book/lecture11.html#solving-pdes-with-the-method-of-weighted-residuals).

The finite element method as defined here is both a method of weighted residuals and a Galerkin method. 

We will start this lecture by considering Poisson's equation

$$
u'' = f, \quad x \in [0, L], \, u(0) = u(L) = 0.
$$

using piecewise linear polynomials as basis functions. The variational Galerkin form is now

$$
(u'', v) = (f, v), \quad \forall \, v \in V_N,
$$

which is modified into

$$
-(u', v') = (f, v) - [u'v]_{x=0}^{x=L}, \quad \forall \, v \in V_N.
$$

Inserting for $u\approx u_N$ and $v=\psi_i$ we get

$$
-\sum_{j=0}^N(\psi'_j, \psi'_i) \hat{u}_j = (f, \psi_i) - (u'_N(L)\psi_N(L) - u'_N(0)\psi_0(0)).
$$(eq-fem-poisson)

The boundary term $(u'_N(L)\psi_N(L) - u'_N(0)\psi_0(0))$ can be used to specify Neumann boundary conditions, and we will get back to that. However, for now we consider only Dirichlet conditions. How can this be implemented? The boundary term contains only the derivatives $u'_N$ and not $u(0)$ or u(L), so this cannot be used. Remember that for Dirichlet conditions the solution is known at the edges, and not unknown. The finite element solution is using Lagrange polynomials that are such that

$$
\psi_j(x_i) = \delta_{ij}.
$$

Hence

$$
u(0) = u_N(x_0) = \sum_{j=0}^N \hat{u}_j \psi_j(x_0) = \psi_0(x_0) \hat{u}_0 = \hat{u}_0,
$$

and 

$$
u(L) = u_N(x_N) = \sum_{j=0}^N \hat{u}_j \psi_j(x_N) = \psi_N(x_N) \hat{u}_N = \hat{u}_N.
$$

So the two expansion coefficients $\hat{u}_0$ and $\hat{u}_N$ are known. That means that we only need to solve {eq}`eq-fem-poisson` for $i \in (1, 2, \ldots, N-1)$. And the boundary term $u'_N(L)\psi_N(L) - u'_N(0)\psi_0(0)$ is only (possibly) nonzero for $i=0$ or $i=N$. So with Dirichlet boundary conditions we can simply neglect the boundary terms! We will get back to this term for Neumann boundary conditions.

To solve {eq}`eq-fem-poisson`, the right hand side is treated exactly as in {ref}`lecture 10 <fem-assembly-vector>`, whereas we need to assemble the stiffness matrix 

$$
s_{ij} = -(\psi'_j, \psi'_i) = \sum_{e=0}^{N_e-1} \int_{\Omega^{(e)}} \psi'_j \psi'_i d\Omega.
$$

We will use the same notation as in lecture 10 and 

$$
s^{(e)}_{ij} = \int_{\Omega^{(e)}} \psi'_j \psi'_i d\Omega
$$

is the element stiffness matrix of shape $(N+1) \times (N+1)$. We also use the same mapping from local to global degrees of freedom, and define a dense element matrix as

$$
\tilde{s}^{(e)}_{rs} = s^{(e)}_{q(e,r), q(e, s)}, \, (r, s) \in (0, 1, \ldots, d)^2 
$$

for finite elements of order $d$. Remember that $d=1$ for linear elements, and then higher order elements ($d>1$) simply uses more nodes within each element. Since this mapping is exactly the same as in lecture 10, we do not repeat how it works here. If you need to be reminded, then see {ref}`the finite element assembly movie<mov-assemble-mass>`.




The finite element stiffness matrix is implemented using a mapping to the reference domain $X\in[-1, 1]$, and the reference basis functions $\psi_{q(e, r)}(x) = \ell_r(X)$

$$
\begin{align*}
\tilde{s}^{(e)}_{rs} &= \int_{\Omega^{(e)}} \frac{d \psi_{q(e, s)}(x)}{dx} \frac{d \psi_{q(e, r)}(x)}{dx} dx, \\
 &= \int_{-1}^1 \frac{d \ell_s(X)}{dX} \frac{dX}{dx} \frac{d \ell_r(X)}{dX} \frac{dX}{dx} \frac{dx}{dX}dX
\end{align*}
$$

With the linear mapping {eq}`eq-affine-map` we get that 

$$
\frac{dx}{dX} = \frac{h}{2}, 
$$

where $h = x_R-x_L$ is the element length. We get

$$
\tilde{s}^{(e)}_{rs} = \int_{-1}^1 \frac{d \ell_s(X)}{dX} \frac{d \ell_r(X)}{dX} \frac{2}{h} dX,
$$

which we write more easily as

$$
\tilde{s}^{(e)}_{rs} = \frac{2}{h} \int_{-1}^1 \ell'_s(X) \ell'_r(X)  dX.
$$

We can assemble the element stiffness matrix once and for all using Sympy:

In [ ]:
import numpy as np
import sympy as sp
from lagrange import Lagrangebasis, Lagrangefunction

x, h = sp.symbols('x,h')
l = Lagrangebasis([-1, 1])
se = lambda r, s: sp.integrate(l[r].diff(x, 1)*l[s].diff(x, 1), (x, -1, 1))
S1e = 2/h*sp.Matrix([[se(0, 0), se(0, 1)],[se(1, 0), se(1, 1)]])
S1e

We can implement a generic routine for higher order elements of order $d$. However, since it is easy to make this implementation for any generic element matrix, we implement instead

$$
\tilde{a}^{(e,m,n)}_{rs} = \left(\frac{h}{2}\right)^{1-(m+n)} \int_{-1}^1 \ell^{(n)}_s(X) \ell^{(m)}_r(X)  dX
$$

In [ ]:
Xj = lambda d: 2*(np.array([sp.Rational(i, d) for i in np.arange(d+1)]))-1 # np.linspace(-1, 1, d+1) only rational
ll = lambda d: Lagrangebasis(Xj(d))
ade = lambda l, r, s, m, n: sp.integrate(l[r].diff(x, m)*l[s].diff(x, n), (x, -1, 1))
def Ade(d=1, m=0, n=0):
    A = np.zeros((d+1, d+1), dtype=object)
    l = ll(d)
    for r in range(d+1):
        for s in range(d+1):
            A[r, s] = ade(l, r, s, m, n)
    return (h/2)**(1-m-n)*sp.Matrix(A)

In order to assemble the global matrix, we need to loop over all elements. This is accomplished as shown below:

In [ ]:
from fem import get_element_boundaries, get_element_length, local_to_global_map

def assemble_generic_matrix(xj, d=1, m=0, n=0):
    N = len(xj)-1
    Ne = N//d
    A = np.zeros((N+1, N+1))
    Ad = Ade(d, m, n)
    for elem in range(Ne):
        hj = get_element_length(xj, elem, d=d)
        s0 = local_to_global_map(elem, d=d)
        A[s0, s0] += np.array(Ad.subs(h, hj), dtype=float)
    return A

N = 4
xj = np.linspace(1, 2, N+1)
S = assemble_generic_matrix(xj, d=1, m=1, n=1)
print(S)

The mass matrix is similarly

In [ ]:
A = assemble_generic_matrix(xj, d=1, m=0, n=0)
print(A)